# Se busca escribir los datos para las progresivas solicitadas


In [44]:
import pandas as pd
import numpy as np
import utm

## Lectura de datos


### Lectura de progresivas


In [45]:
# read progresivas.xlsx data

# read data from excel file
df = pd.read_excel("./data/progresivas.xlsx", sheet_name="Hoja1")

df

,0
0,25
1,50
2,75
3,100
4,125
5,150
6,175
7,200
8,225
9,250


In [46]:
progresivas = df[0].tolist()
progresivas

[25, 50, 75, 100, 125, 150, 175, 200, 225, 250, 275, 300]

### Lectura de los datos xyz del archivo json


In [47]:
# read data from json file
df = pd.read_json("./output/joined_data.json")
df.head()

,longitud,latitud,x,y,zone_number,zone_letter,elevacion,distancia,distancia_acumulada
0,-71.508228,-15.525022,230941.827679,8.282026e+06,19,L,3811.707520,0.000000,0.000000
1,-71.507553,-15.525043,231014.292173,8.282024e+06,19,L,3811.989258,72.346343,72.346343
2,-71.507130,-15.525102,231059.779810,8.282018e+06,19,L,3811.907227,45.809196,118.155539
3,-71.506743,-15.525320,231101.550085,8.281995e+06,19,L,3807.909912,47.948586,166.104125
4,-71.506286,-15.525523,231150.935478,8.281973e+06,19,L,3806.519775,53.980618,220.084743


### Verificacion de datos


In [48]:
def verificar_orden(lista):
    if all(lista[i] <= lista[i + 1] for i in range(len(lista) - 1)):
        print("Ok")
    else:
        raise ValueError("La lista no está ordenada ascendentemente")


# Ejemplo de uso

try:
    verificar_orden(progresivas)
except ValueError as e:
    print(f"Error: {e}")

Ok


In [49]:
def interpolador(x, ti, tj):
    xi, xj = ti[0], tj[0]
    yi, yj = ti[1], tj[1]

    return yi + (yj - yi) * (x - xi) / (xj - xi)

In [58]:
def interpolador_progresivas(progresiva, data):
    low_index = 0
    high_index = 0

    response = {
        "progresiva": progresiva,
        "x": 0,
        "y": 0,
        "latitude": 0,
        "longitude": 0,
        "elevation": 0,
        "zone_number": 0,
        "zone_letter": "",
    }

    for i in range(data.shape[0]):
        if data.iloc[i]["distancia_acumulada"] <= progresiva:
            low_index = i
        if data.iloc[i]["distancia_acumulada"] >= progresiva:
            high_index = i
            break

    # Para el caso de que ambos puntos sean de la misma zona

    if (
        data.iloc[low_index]["zone_number"] == data.iloc[high_index]["zone_number"]
        and data.iloc[low_index]["zone_letter"] == data.iloc[high_index]["zone_letter"]
    ):
        x_coordinate = interpolador(
            progresiva,
            (
                data.iloc[low_index]["distancia_acumulada"],
                data.iloc[low_index]["x"],
            ),
            (
                data.iloc[high_index]["distancia_acumulada"],
                data.iloc[high_index]["x"],
            ),
        )

        y_coordinate = interpolador(
            progresiva,
            (
                data.iloc[low_index]["distancia_acumulada"],
                data.iloc[low_index]["y"],
            ),
            (
                data.iloc[high_index]["distancia_acumulada"],
                data.iloc[high_index]["y"],
            ),
        )

        latitude, longitude = utm.to_latlon(
            x_coordinate,
            y_coordinate,
            data.iloc[low_index]["zone_number"],
            data.iloc[low_index]["zone_letter"],
        )

        response["x"] = x_coordinate
        response["y"] = y_coordinate
        response["latitude"] = latitude
        response["longitude"] = longitude
        response["zone_number"] = data.iloc[low_index]["zone_number"]
        response["zone_letter"] = data.iloc[low_index]["zone_letter"]

    else:
        latitude = interpolador(
            progresiva,
            (
                data.iloc[low_index]["distancia_acumulada"],
                data.iloc[low_index]["latitud"],
            ),
            (
                data.iloc[high_index]["distancia_acumulada"],
                data.iloc[high_index]["latitud"],
            ),
        )

        longitude = interpolador(
            progresiva,
            (
                data.iloc[low_index]["distancia_acumulada"],
                data.iloc[low_index]["longitud"],
            ),
            (
                data.iloc[high_index]["distancia_acumulada"],
                data.iloc[high_index]["longitud"],
            ),
        )

        x_coordinate, y_coordinate, zone_number, zone_letter = utm.from_latlon(
            latitude, longitude
        )

        response["x"] = x_coordinate
        response["y"] = y_coordinate
        response["latitude"] = latitude
        response["longitude"] = longitude
        response["zone_number"] = data.iloc[low_index]["zone_number"]
        response["zone_letter"] = data.iloc[low_index]["zone_letter"]

    elevation = interpolador(
        progresiva,
        (
            data.iloc[low_index]["distancia_acumulada"],
            data.iloc[low_index]["elevacion"],
        ),
        (
            data.iloc[high_index]["distancia_acumulada"],
            data.iloc[high_index]["elevacion"],
        ),
    )

    response["elevation"] = elevation

    return response

    # print(low_index, high_index)

In [59]:
interpolador_progresivas(160, df)

{'progresiva': 160,
 'x': 231096.2324938025,
 'y': 8281997.759639837,
 'latitude': -15.525292034932116,
 'longitude': -71.50679269053553,
 'elevation': 3808.418792717243,
 'zone_number': 19,
 'zone_letter': 'L'}

In [63]:
final=[]

for progresiva in progresivas:
    final.append(interpolador_progresivas(progresiva, df))

## Imprimiendo los resultados en una hoja excel

In [65]:
# Write final in excel file

df = pd.DataFrame(final)
df.to_excel("output/progresivas_data.xlsx", index=False)